In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os 
import copy

In [20]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '/content/drive/MyDrive/cakd7/강의/m10_딥러닝응용/hymenoptera_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [22]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict()) # 모델의 현재 상태에 대한 참조 반환
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # 모델을 학습 모드로 설정
            else:
                model.eval()   # 모델을 평가 모드로 설정

            running_loss = 0.0
            running_corrects = 0

            # 데이터를 반복
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # 매개변수 경사도를 0으로 설정
                optimizer.zero_grad()

                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1) # if you want to find the maximum along the rows, use dim=1.
                    loss = criterion(outputs, labels)

                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                
                running_loss += loss.item() * inputs.size(0)
                # loss.item() contains the loss of the entire mini-batch,  
                
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # 가장 나은 모델 가중치를 불러옴
    model.load_state_dict(best_model_wts)
    return model

In [23]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features

model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# 7 에폭마다 0.1씩 학습률 감소
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [24]:
model_ft = train_model(model_ft,criterion,optimizer_ft,exp_lr_scheduler, num_epochs=25 )

Epoch 0/24
----------
train Loss: 0.6112 Acc: 0.6803
val Loss: 0.1999 Acc: 0.9281

Epoch 1/24
----------
train Loss: 0.5576 Acc: 0.7582
val Loss: 0.2771 Acc: 0.9020

Epoch 2/24
----------
train Loss: 0.5081 Acc: 0.8238
val Loss: 0.2766 Acc: 0.8889

Epoch 3/24
----------
train Loss: 0.4831 Acc: 0.8074
val Loss: 0.2954 Acc: 0.8693

Epoch 4/24
----------
train Loss: 0.3860 Acc: 0.8566
val Loss: 0.2143 Acc: 0.9085

Epoch 5/24
----------
train Loss: 0.5947 Acc: 0.7582
val Loss: 0.3209 Acc: 0.9020

Epoch 6/24
----------
train Loss: 0.6048 Acc: 0.7705
val Loss: 0.2810 Acc: 0.9346

Epoch 7/24
----------
train Loss: 0.4922 Acc: 0.8361
val Loss: 0.3348 Acc: 0.9020

Epoch 8/24
----------
train Loss: 0.5299 Acc: 0.8156
val Loss: 0.2205 Acc: 0.9085

Epoch 9/24
----------
train Loss: 0.4245 Acc: 0.8197
val Loss: 0.1919 Acc: 0.9216

Epoch 10/24
----------
train Loss: 0.3045 Acc: 0.8607
val Loss: 0.2475 Acc: 0.8824

Epoch 11/24
----------
train Loss: 0.3336 Acc: 0.8607
val Loss: 0.1889 Acc: 0.9346

Ep